In [1]:
import cv2
import os
from PIL import Image,ImageOps
import numpy as np
image_directory = 'Datasets/'

dataset = []
label = []
INPUT_SIZE = 64
no_tumor_brain = os.listdir(image_directory + 'no/')
for i, image_name in enumerate(no_tumor_brain):
    image = cv2.imread(image_directory + '/no/' + image_name)
    image = Image.fromarray(image,'RGB')
    max_side = max(image.size)
    # Tính toán padding
    left_padding = (max_side - image.width) // 2
    top_padding = (max_side - image.height) // 2
    right_padding = max_side - image.width - left_padding
    bottom_padding = max_side - image.height - top_padding

    # Thêm padding để tạo hình vuông
    image = ImageOps.expand(image, (left_padding, top_padding, right_padding, bottom_padding), fill=(0, 0, 0)) # Phần thêm sẽ để màu đen

    # Tùy chọn: Thay đổi kích thước hình ảnh thành hình vuông với kích thước mong muốn
    final_size = (INPUT_SIZE, INPUT_SIZE)
    image = image.resize(final_size)
    dataset.append(np.array(image))
    label.append(0)


yes_tumor_brain = os.listdir(image_directory + 'yes/')
for i, image_name in enumerate(yes_tumor_brain):
    image = cv2.imread(image_directory + '/yes/' + image_name)
    image = Image.fromarray(image,'RGB')
    max_side = max(image.size)
    # Tính toán padding
    left_padding = (max_side - image.width) // 2
    top_padding = (max_side - image.height) // 2
    right_padding = max_side - image.width - left_padding
    bottom_padding = max_side - image.height - top_padding

    # Thêm padding để tạo hình vuông
    image = ImageOps.expand(image, (left_padding, top_padding, right_padding, bottom_padding), fill=(0, 0, 0)) # Phần thêm sẽ để màu đen

    # Tùy chọn: Thay đổi kích thước hình ảnh thành hình vuông với kích thước mong muốn
    final_size = (INPUT_SIZE, INPUT_SIZE)
    image = image.resize(final_size)
    dataset.append(np.array(image))
    label.append(1)

print(dataset[0])
print(label[0])

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0


In [2]:
dataset = np.array(dataset)
label = np.array(label)

In [3]:
from sklearn.model_selection import train_test_split
 

In [4]:
x_train, x_test, y_train, y_test = train_test_split(dataset,label, test_size=0.2, random_state=42)


In [5]:
from tensorflow import keras
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Activation,Dropout, Flatten, Dense

In [6]:
x_train = normalize(x_train,axis=1)
x_test = normalize(x_test,axis=1)


In [7]:
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))





f:\AIi\myenvs\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(x_train,y_train, 
        batch_size= 36,
        verbose=1,
        epochs=20,
        #validation_data=(x_test,y_test), #
        validation_split = 0.2,
        shuffle=False)

model.save("BrainTumor10epochs.h5")

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.5957 - loss: 0.6517 - val_accuracy: 0.7729 - val_loss: 0.5381
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.7491 - loss: 0.5242 - val_accuracy: 0.8125 - val_loss: 0.4630
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.8050 - loss: 0.4453 - val_accuracy: 0.8125 - val_loss: 0.4014
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.8349 - loss: 0.3816 - val_accuracy: 0.8542 - val_loss: 0.3396
Epoch 5/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.8737 - loss: 0.3179 - val_accuracy: 0.8813 - val_loss: 0.2821
Epoch 6/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.8981 - loss: 0.2679 - val_accuracy: 0.9062 - val_loss: 0.2466
Epoch 7/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.9162 - loss: 0.2214 - val_accuracy: 0.9187 - val_loss: 0.2060
Epoch 8/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.9395 - loss: 0.1629 - val_accuracy: 0.9396 - v

In [10]:
y_pred = model.predict(x_test)
y_pred_classes = (y_pred > 0.5).astype("int32")
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred_classes)
print("F1 Score: ", f1)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step
F1 Score:  0.9630931458699473


In [11]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred_classes, target_names=['0', '1'])
print(report)

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       313
           1       0.97      0.95      0.96       287

    accuracy                           0.96       600
   macro avg       0.97      0.96      0.96       600
weighted avg       0.97      0.96      0.96       600

